<a href="https://colab.research.google.com/github/AlbertoB12/AIVideoShortsGenerator/blob/main/AIVideoShortsGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### Script to automatically create AI videos for Youtube (Shorts), Instagram (Reels) and TikTok ####
print("Welcome to the AI Video Generation script for social media!")
### Packages, imports and keys ###
print("Installing and importing packages and resources.")
!pip install --upgrade -q pandas
!pip install --upgrade -q networkx
!pip install -q colab-xterm
#!pip install -q ollama
!pip install -q moviepy
!pip install -q librosa
!pip install -q soundfile
!pip install -q edge-tts
!apt-get install -y -qq pciutils > /dev/null 2>&1
!apt-get install -y -qq lshw > /dev/null 2>&1
!pip install -q TTS
!pip install -q google-api-python-client
!pip install -q google-auth-httplib2
!pip install -q google-auth-oauthlib
!pip install -q tweepy
import psutil, requests, tempfile, torch, os, re, tweepy, time, random, warnings, librosa, soundfile, asyncio, nest_asyncio, edge_tts #, ollama
!apt-get install -y imagemagick > /dev/null 2>&1
!if [ -f /etc/ImageMagick-6/policy.xml ]; then sudo sed -i 's/rights="none"/rights="read|write"/g' /etc/ImageMagick-6/policy.xml; fi
!if [ -f /etc/ImageMagick-7/policy.xml ]; then sudo sed -i 's/rights="none"/rights="read|write"/g' /etc/ImageMagick-7/policy.xml; fi
!head -n 20 /etc/ImageMagick-6/policy.xml
!head -n 20 /etc/ImageMagick-7/policy.xml
os.environ['IMAGEMAGICK_BINARY'] = '/usr/bin/convert'
import numpy as np
from TTS.api import TTS
from openai import OpenAI
from moviepy.editor import VideoFileClip, AudioFileClip, TextClip, CompositeVideoClip, vfx
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.colab import runtime
from IPython.display import Javascript
warnings.filterwarnings("ignore")
nest_asyncio.apply()
"""
%load_ext colabxterm
!curl -fsSL https://ollama.com/install.sh | bash
!nohup ollama serve &
!ollama pull llama3.1:8b
"""
## Pixabay keys
pixabay_key = ''
## X keys
api_key = ''
api_secret_key = ''
access_token = ''
access_token_secret = ''
bearer_token = ''
# Set up authentication with X API
auth = tweepy.OAuth1UserHandler(
    consumer_key=api_key,
    consumer_secret=api_secret_key,
    access_token=access_token,
    access_token_secret=access_token_secret
)
# Authentication with OpenAI (Deepseek - OpenRouter)
client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="",
)

### Functions ###
## def create_and_upload_video(desired_topic, standard_topic, saving_path, voice_model, client_secrets)
# Main function to run the whole script
# Gets strings with desired_topic, standard_topic, saving_path, voice_model, client_secrets
# Generates script, gets suitable video, generates voiceover, edits video, publishes video
def create_and_upload_video(desired_topic, standard_topic, saving_path, voice_model, client_secrets):
  ## Step 1: Script creation
  print("Starting step 1: Script creation.")
  # Generate script
  script = generate_script(desired_topic)

  ## Step 2: Video creation
  print("Starting step 2: Video creation.")
  # Download and save video
  get_video(desired_topic, standard_topic, saving_path)

  ## Step 3: Voiceover
  print("Starting step 3: Voiceover.")
  # Create and save voiceover
  loop = asyncio.get_event_loop()
  loop.run_until_complete(get_voiceover(script, saving_path, voice_model))

  ## Step 4: Video editing
  print("Starting step 4: Video editing.")
  # Edit video (merge video and voiceover, create subtitles)
  edit_video(script, saving_path)

  ## Step 5: Publishing
  print("Starting step 5: Publishing.")
  # Generate parameters for the video: title, description and tags
  print("Generating video parameters.")
  title, description, tags = get_video_parameters(script)
  # Uploads
  print("Uploading video.")
  upload_video(title, description, tags, saving_path)

  ## Step 6: Finished!
  print("All done!")
  # Disconnect from Google Colab
  print("Finalizing execution.")
  display(Javascript('google.colab.kernel.disconnect()'))

## def generate_script(desired_topic)
# Function to generate script and topic for the video
# Gets a str with the deseired topic
# Returns str with script for the video
def generate_script(desired_topic):
  # If user introduced a topic
  if desired_topic != " " or desired_topic != "":
    # Generate script
    script = client.chat.completions.create(
      model="deepseek/deepseek-r1:free",
      messages=[
        {
          "role": "user",
          "content": f"""Generate a concise, high-impact script that generates dopamine in users of social media for a short video with the topic '{desired_topic}'. Keep it between 80 and 120 words. The tone should be powerful, inspiring, and direct. Use short sentences and impactful phrasing. The script should encourage the values of the topic. No introduction, explanation, instructions for video or audio or ending indications, just the script as plain text, without asteriscs, etc.."""
        }
      ]
    ).choices[0].message.content
    script_clean = script.replace('"', '')
    # Return script
    return script_clean
  # If user did not introduce a topic
  else:
    print("No topic added! Please add a topic and start the execution again.")
    # Delete runtime
    display(Javascript('google.colab.kernel.disconnect()'))

## def split_script(script, max_chars=200, min_chars=50)
# Function to split script into chunks without breaking words or sentences
# Gets str with script to be splitted and 2 int with max. und min. characters to form the chuncks
# Returns str with chuncks of the script
def split_script(script, max_chars=200, min_chars=50):
  sentences = re.split(r'(?<=[.!?])\s+', script)
  chunks = []
  current_chunk = ""

  for sentence in sentences:
    if len(current_chunk) + len(sentence) <= max_chars:
      current_chunk += (" " if current_chunk else "") + sentence
    else:
      if len(current_chunk.strip()) >= min_chars:
        chunks.append(current_chunk.strip())
        current_chunk = sentence
      else:
        current_chunk += " " + sentence  # Merge short chunk

  if current_chunk:
    chunks.append(current_chunk.strip())

  # Final pass: Merge very short chunks
  final_chunks = []
  for chunk in chunks:
    if final_chunks and len(chunk) < min_chars:
      final_chunks[-1] += " " + chunk
    elif chunk:
      final_chunks.append(chunk)

  return final_chunks

## def get_video(desired_topic, standard_topic, saving_path)
# Funciton to download and save a video
# Gets a str with the desired topic, a str with the safety topic and a str with the saving path
# Saves the downloaded video
def get_video(desired_topic, standard_topic, saving_path):
    base_url = "https://pixabay.com/api/videos/"

    def simple_quote(s):
        return s.replace(" ", "+")

    topics_to_search = [desired_topic, standard_topic]

    for topic in topics_to_search:
        encoded_query = simple_quote(topic)
        url = f"{base_url}?key={pixabay_key}&q={encoded_query}"

        try:  # Add a try-except block for the request
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

            data = response.json()

            if data["hits"]:
                selected_hit = random.choice(data["hits"])
                video_url = selected_hit["videos"]["medium"]["url"]
                video_path = f"{saving_path}Video.mp4"
                with open(video_path, 'wb') as f:
                    f.write(requests.get(video_url).content)
                print(f"Video downloaded successfully to: {video_path} (using topic: {topic})")
                return True

            else:
                print(f"No video results found for '{topic}' (even though request was OK).")

        except requests.exceptions.RequestException as e: # Catch request errors
            print(f"Error fetching data for '{topic}': {e}")  # Print the actual error from requests
            # Delete runtime
            display(Javascript('google.colab.kernel.disconnect()'))


        except (KeyError, ValueError) as e: # Handle JSON decoding errors
            print(f"Error parsing JSON response for '{topic}': {e}. Response content: {response.text}")
            # Delete runtime
            display(Javascript('google.colab.kernel.disconnect()'))

    print("No suitable videos found for any topic.")
    # Delete runtime
    display(Javascript('google.colab.kernel.disconnect()'))

## async def get_voiceover(script, saving_path, voice_model)
# Function to create and save voiceover for the video
# Gets a str with the script for the voiceover, a str with the saving path and a str with the name of the voiceover model
# Saves the created voiceover
async def get_voiceover(script, saving_path, voice_model):
  try:
    # Get device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Initialize TTS model
    #tts = TTS(model_name=voice_model, progress_bar=False).to(device)

    # List to store generated audio file paths
    audio_files = []

    temp_path = f"{saving_path}voice_part_0.wav"
    #tts.tts_to_file(text=script, file_path=temp_path)
    tts = edge_tts.Communicate(script, voice="en-AU-WilliamNeural")
    await tts.save(temp_path)
    audio_files.append(temp_path)

    # Merge audio files if multiple parts exist
    final_audio_path = f"{saving_path}Voiceover.wav"
    if len(audio_files) > 1:
      combined_audio = []
      sample_rate = None

      for audio_file in audio_files:
        audio, sr = librosa.load(audio_file, sr=None)
        if sample_rate is None:
          sample_rate = sr
        combined_audio.append(audio)
        os.remove(audio_file)  # Clean up temporary files

      # Concatenate audio and save
      merged_audio = np.concatenate(combined_audio)
      sf.write(final_audio_path, merged_audio, sample_rate)
    else:
      # If only one part, rename it as final output
      os.rename(audio_files[0], final_audio_path)

  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

## def edit_video(script, saving_path):
# Function to merge video and voiceover, creating the final video to be uploaded to social media
# Access to video and voiceover
# Gets str with script for the video and a str with the saving path
# Creates the final video
def edit_video(script, saving_path):
  try:
    # Load video and voiceover
    video = VideoFileClip(f"{saving_path}Video.mp4")
    voiceover = AudioFileClip(f"{saving_path}Voiceover.wav")
  except Exception as e:
    print(f"Video or audio not found. An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Get the duration of the video and the voiceover
  try:
    video_duration = video.duration
    voiceover_duration = voiceover.duration
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Adjust video duration to match voiceover
  try:
    if voiceover_duration < video_duration:
      video = video.subclip(0, voiceover_duration)
    elif voiceover_duration > video_duration:
      loop_count = int(voiceover_duration // video_duration) + 1
      video = video.fx(vfx.loop, n=loop_count).subclip(0, voiceover_duration)
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Split the script into meaningful parts
  try:
    script_parts = split_script(script, max_chars=200)
    sentence_duration = voiceover_duration / len(script_parts)
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  try:
    subtitles = []
    for i, sentence in enumerate(script_parts):
      start_time = i * sentence_duration
      end_time = (i + 1) * sentence_duration
      subtitles.append({"text": sentence, "start": start_time, "end": end_time})
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Create text clips for each subtitle
  try:
    text_clips = []

    for subtitle in subtitles:
      # Calculate duration
      duration = subtitle['end'] - subtitle['start']

      # Bounce effect function - NOW IN THE MIDDLE
      def bounce_effect(t):
        return ('center', video.h / 2 - abs(10 * np.sin(2 * np.pi * t))) # Vertical center is video.h / 2

      # Create main TextClip with enhanced style
      txt_clip = TextClip(
        subtitle['text'],
        fontsize=75,
        color='white',
        stroke_color='black',
        stroke_width=4,
        align='center',
        method='caption',
        size=(video.w * 0.9, None)
      ).set_duration(duration).set_position(bounce_effect).fadein(0.2).fadeout(0.2)

      # Create shadow effect (optional)
      shadow = txt_clip.set_opacity(0.5).set_position(("center", video.h / 2 + 50)) # Shadow a bit below center

      # Combine shadow and text
      final_txt_clip = CompositeVideoClip([shadow, txt_clip]) # Or just use txt_clip alone: final_txt_clip = txt_clip

      # Set subtitle start time
      final_txt_clip = final_txt_clip.set_start(subtitle['start'])

      text_clips.append(final_txt_clip)
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Combine the video and subtitles
  try:
    video_with_subtitles = CompositeVideoClip([video] + text_clips)
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Set the audio of the video to the voiceover
  try:
    final_video = video_with_subtitles.set_audio(voiceover)
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

  # Export the final video
  try:
    final_video.write_videofile(f"{saving_path}final_video.mp4", codec="libx264", audio_codec="aac")
  except Exception as e:
    print(f"An error occurred: '{e}'")
    print("Finalizing execution.")
    display(Javascript('google.colab.kernel.disconnect()'))

## def generate_video_title(script)
# Function to generate a title for the video before uploading it to social media
# Gets str with script of the video
# Returns str with title for the video
def generate_video_title(script):
  # Generate title
  title = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
      {
        "role": "user",
        "content": f"""Generate an engaging and attention-grabbing title for a social media video based on the following script: '{script}'.
The title should be concise, impactful, and relevant to the content, suitable for platforms like YouTube, Instagram, and TikTok. It should also generate dopamine in the users. Do not generate introductions or explanations, just the title."""
      }
    ]
  ).choices[0].message.content
  return title

## def generate_video_description(script)
# Function to generate a description for the video before uploading it to social media
# Gets str with script of the video
# Returns str with description for the video
def generate_video_description(script):
  # Generate description
  description = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
      {
        "role": "user",
        "content": f"""Generate an engaging and attention-grabbing description for a social media video based on the following script: '{script}'.
The description should be concise, impactful, and relevant to the content, suitable for platforms like YouTube, Instagram, and TikTok. It should also generate dopamine in the users. Do not generate introductions or explanations, just the description."""
      }
    ]
  ).choices[0].message.content
  return description

## def generate_video_tags(script)
# Function to generate tags for the video before uploading it to social media
# Gets str with script of the video
# Returns str with tags for the video
def generate_video_tags(script):
  # Generate tags
  tags = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[
      {
        "role": "user",
        "content": f"""Generate an engaging and attention-grabbing title for a social media video based on the following script: '{script}'.
The title should be concise, impactful, and relevant to the content, suitable for platforms like YouTube, Instagram, and TikTok. It should also generate dopamine in the users. Do not generate introductions or explanations, just the title."""
      }
    ]
  ).choices[0].message.content
  return tags

## def get_video_parameters (script)
# Function to generate once in the same function the title, description and tags for the video
# Gets str with the script of the video
# Returns 3 str with title, description and tags for the video
def get_video_parameters(script):
  # Generate title
  title = generate_video_title(script)
  # Generate description
  description = generate_video_description(script)
  # Generate tags
  tags = generate_video_tags(script)

  return title, description, tags

## def upload_youtube(title, description, tags, saving_path)
# Function to upload video to YouTube
# Gets strings with title, description, tags and saving path of the video
# Uploads the video to YouTube
def upload_youtube(title, description, tags, saving_path):
  print("Uploading video to YouTube.")
  # Set up authentication
  scopes = ['https://www.googleapis.com/auth/youtube.upload']
  flow = InstalledAppFlow.from_client_secrets_file(client_secrets, scopes)
  credentials = flow.run_local_server(port=0)

  # Build the YouTube API client
  youtube = build('youtube', 'v3', credentials=credentials)

  title = f"{title} #Shorts"

  # Create a request to upload the Short
  request = youtube.videos().insert(
    part="snippet,status",
    body={
      "snippet": {
      "title": title,
      "description": f"{description} Video from: https://pixabay.com/",
      "tags": tags,
      "categoryId": '27',
    },
    "status": {
      "privacyStatus": "public",
    }
    },
    media_body=MediaFileUpload(f"{saving_path}final_video.mp4", chunksize=-1, resumable=True)
  )

  # Execute the request
  response = request.execute()
  print(f"YouTube upload successful! Video ID: {response['id']}")

## upload_x(title, description, tags, saving_path)
# Function to upload video to X
# Gets strings with title, tags and saving path of the video
# Uploads the video to X
def upload_x(title, tags, saving_path):
  print("Uploading video to X.")
  # Create API object
  api = tweepy.API(auth)

  # Step 1: Upload the video to X as media
  media = api.media_upload(f"{saving_path}final_video.mp4")

  # Step 2: Post a tweet with the uploaded video
  # Text that will accompany the video
  status = f"{title} {tags} Video from: https://pixabay.com/"
  # Upload
  tweet = api.update_status(status=status, media_ids=[media.media_id])
  print(f"Successfully uploaded video to X! Tweet ID: {tweet.id}")

def upload_facebook(title, description, tags, saving_path):
  print("Uploading video to Facebook.")
  # Will be addded in the future

def upload_instagram(title, description, tags, saving_path):
  print("Uploading video to Instagram.")
  # Will be addded in the future

def upload_tiktok(title, description, tags, saving_path):
  print("Uploading video to TikTok.")
  # Will be addded in the future

## def upload_video(title, description, tags, saving_path)
# Function to upload the final video to YouTube, TikTok, Instagram and X
# Gets a str with the title of the video, a str with the description of the video, a str with the tags and a str with the saving path
# Uploads the video to the different platforms
def upload_video(title, description, tags, saving_path):
  # Upload to YouTube
  upload_youtube(title, description, tags, saving_path)

  # Upload to X
  upload_x(title, tags, saving_path)

  # Upload to Facebook
  #upload_facebook(title, description, tags, saving_path)

  # Upload to Instagram
  #upload_instagram(title, description, tags, saving_path)

  # Upload to TikTok
  #upload_tiktok(title, description, tags, saving_path)


### Code ###
try:
  ## Preparation
  print("Starting preparation of the whole process.")
  # Variables to decide, if desired, the topic and 'safety' topic of the video
  desired_topic = ""  # If desired, add a topic for the video. If not, leave as it is -> " "
  standard_topic = ""  # Standard topic for searching a video, if the topic generated by the AI model doesn't result on a video  # Always has to have a value
  saving_path = ""  # Path to temporary save materials
  voice_model = 'tts_models/en/ljspeech/tacotron2-DDC'  # Desired voice model
  client_secrets = ''  # Path to the client_secrets.json for YouTube-upload
  print(f"Chosen topics:\nDesired topic: '{desired_topic}'.\nSafety topic: '{standard_topic}'.\n\nChosen directory to temporary save materials: '{saving_path}'.\n\nChosen voice model: {voice_model}.\n\nChosen path for client.secrets.json for YouTube: {client_secrets}.")
  # Run main function with all sub-functions to create and upload video
  for execution in range(1):  # Change number with number of videos you want to generate
    create_and_upload_video(desired_topic, standard_topic, saving_path, voice_model, client_secrets)
except Exception as e:
  print(f"An error occurred: '{e}'")
  print("Finalizing execution.")
  # Disconnect from Google Colab
  display(Javascript('google.colab.kernel.disconnect()'))


Welcome to the AI Video Generation script for social media!
Installing and importing packages and resources.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 21.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 65.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8

  if event.key is 'enter':



Starting preparation of the whole process.
Chosen topics:
Desired topic: 'Stoicism'.
Safety topic: 'motivation'.

Chosen directory to temporary save materials: '/content/drive/MyDrive/Documentos/Projects/Automatic videos/Material/'.

Chosen voice model: tts_models/en/ljspeech/tacotron2-DDC.

Chosen path for client.secrets.json for YouTube: /content/drive/MyDrive/Documentos/Projects/Automatic videos/client_secrets.json.
Starting step 1: Script creation.
Starting step 2: Video creation.
No video results found for 'Stoicism' (even though request was OK).
Video downloaded successfully to: /content/drive/MyDrive/Documentos/Projects/Automatic videos/Material/Video.mp4 (using topic: motivation)
Starting step 3: Voiceover.
Starting step 4: Video editing.
Moviepy - Building video /content/drive/MyDrive/Documentos/Projects/Automatic videos/Material/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Documentos/Projects/Automatic videos/Material/final_video.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Documentos/Projects/Automatic videos/Material/final_video.mp4
Starting step 5: Publishing.
Generating video parameters.
Uploading video.
Uploading video to YouTube.
An error occurred: 'could not locate runnable browser'
Finalizing execution.


<IPython.core.display.Javascript object>